In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r --recursive "/content/drive/My Drive/data.zip" "/content"
!unzip data.zip

In [1]:
!pip install keras_efficientnets
!pip install keras_applications

from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras_efficientnets import EfficientNetB5, EfficientNetB0
from keras import backend as K
from keras import applications, Sequential, optimizers
from keras.layers import Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt
import numpy as np
import time
from os.path import exists
from os import makedirs
import os

     |████████████████████████████████| 51kB 4.5MB/s 


In [ ]:
# Xception
# InceptionV3
# InceptionResNetV2
# ResNet50
# NASNetLarge
# EfficientNetB0
# EfficientNetB5

# 1 sigmoid   2 softmax

In [3]:
IMG_SIZE  = 160


base_model =   applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
print(base_model.input , base_model.output)


add_model = base_model.output
add_model = Flatten()(add_model)
add_model = Dropout(0.5)(add_model)
add_model = Dense(512, activation='relu')(add_model)
add_model = Dropout(0.4)(add_model)
add_model = Dense(256, activation='relu')(add_model)
add_model = Dropout(0.4)(add_model)
add_model = Dense(128, activation='relu')(add_model)
add_model = Dropout(0.4)(add_model)
add_model = Dense(2, activation='softmax')(add_model)

model = Model(inputs = base_model.input, outputs = add_model)
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
model.summary()

Tensor("input_2:0", shape=(None, 160, 160, 3), dtype=float32) Tensor("conv5_block3_out/Relu_1:0", shape=(None, 5, 5, 2048), dtype=float32)
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 166, 166, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 80, 80, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 80, 80, 

In [ ]:
import os
import shutil

try:
    shutil.rmtree("/content/data/train/.ipynb_checkpoints") 
except:
    pass
try:
    shutil.rmtree("/content/data/test/.ipynb_checkpoints") 
except:
    pass
try:
    shutil.rmtree("/content/data/val/.ipynb_checkpoints") 
except:
    pass

In [ ]:
# locally from desktop path
if "os" in os.environ:
    train_folder = "/deepfake_videos/data/train"
    test_folder = "/deepfake_videos/data/test"
    val_folder = "/deepfake_videos/data/val"

# in colab
else:
    train_folder = "data/train/"
    test_folder = "data/test/"
    val_folder = "data/val/"
    
    
    
datagenTrain = ImageDataGenerator(
    rescale=1.0 / 255.0,
    horizontal_flip=True,
    # rotation_range=30,
    # zoom_range=0.15,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.15,
    # fill_mode="nearest",
)

datagenTest = ImageDataGenerator(rescale=1.0 / 255.0)
datagenVal = ImageDataGenerator(rescale=1.0 / 255.0)


# prepare an iterators for each dataset
train_it = datagenTrain.flow_from_directory(train_folder,
                                            target_size=(160, 160), 
                                            batch_size = 64,
                                            class_mode='binary')
test_it = datagenTest.flow_from_directory(test_folder,
                                          target_size=(160, 160), 
                                          batch_size = 64,
                                          class_mode='binary')
val_it = datagenVal.flow_from_directory(val_folder,
                                          target_size=(160, 160), 
                                          batch_size = 64,
                                          class_mode='binary')


# confirm the iterator works
train_X, train_Y = train_it.next()
print('Batch shape=%s, %s, min=%.3f, max=%.3f' % (train_X.shape, train_Y.shape, train_X.min(), train_X.max()))

test_X, test_Y = test_it.next()
print('Batch shape=%s, %s, min=%.3f, max=%.3f' % (test_X.shape, test_Y.shape, test_X.min(), test_X.max()))

val_X, val_Y = val_it.next()
print('Batch shape=%s, %s, min=%.3f, max=%.3f' % (val_X.shape, val_Y.shape, val_X.min(), val_X.max()))

Found 227190 images belonging to 2 classes.
Found 28000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Batch shape=(64, 160, 160, 3), (64,), min=0.000, max=1.000
Batch shape=(64, 160, 160, 3), (64,), min=0.000, max=1.000
Batch shape=(64, 160, 160, 3), (64,), min=0.000, max=1.000


In [ ]:
steps_per_epoch = train_it.n//train_it.batch_size
print(train_it.n, train_it.batch_size, steps_per_epoch)

validation_steps  = val_it.n//val_it.batch_size
print(val_it.n,   val_it.batch_size,   validation_steps)

history = model.fit_generator(generator = train_it,
                    steps_per_epoch = steps_per_epoch,
                    validation_data = val_it,
                    validation_steps = validation_steps,
                    epochs=100,
                    verbose = 1
)

227190 64 3549
20000 64 312
Epoch 1/100
3549/3549 [==============================] - 1288s 363ms/step - loss: 0.7481 - accuracy: 0.4995 - val_loss: 0.6934 - val_accuracy: 0.5005
Epoch 2/100
3549/3549 [==============================] - 1292s 364ms/step - loss: 0.7010 - accuracy: 0.4993 - val_loss: 0.6933 - val_accuracy: 0.4999
Epoch 3/100
3549/3549 [==============================] - 1292s 364ms/step - loss: 0.6970 - accuracy: 0.5017 - val_loss: 0.6933 - val_accuracy: 0.5009
Epoch 4/100
3549/3549 [==============================] - 1292s 364ms/step - loss: 0.6956 - accuracy: 0.5018 - val_loss: 0.6932 - val_accuracy: 0.5009
Epoch 5/100
3549/3549 [==============================] - 1290s 364ms/step - loss: 0.6949 - accuracy: 0.5001 - val_loss: 0.6932 - val_accuracy: 0.5009
Epoch 6/100
3549/3549 [==============================] - 1290s 364ms/step - loss: 0.6945 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.5004
Epoch 7/100
3549/3549 [==============================] - 1292s 364ms/ste

In [ ]:
loss = model.evaluate_generator(test_it, steps=24)

In [ ]:
yhat = model.predict_generator(test_it, steps=24)